##### Import statements

In [1]:
import xml.etree.ElementTree as ET
import spacy
from spacy.lang.zh.examples import sentences

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

import torch
from torch.nn.utils.rnn import pad_sequence # not from scratch
from torch.utils.data import TensorDataset, DataLoader

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import random


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/laurah/opt/miniconda3/envs/iw_env/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/laurah/opt/miniconda3/envs/iw_env/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/laurah/opt/miniconda3/envs/iw_env/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739,

##### Parse XML File

In [2]:
import xml.etree.ElementTree as ET

# Parse the XML file for sentence alignments
tree = ET.parse('/Users/laurah/Desktop/PRINCETON/junior fall/IW/iw_2/en-zht.txt/GlobalVoices.en-zht.xml')
root = tree.getroot()

alignments = []
for link in root.findall('.//link'):
    x_targets = link.get('xtargets')
    eng_idx, mand_idx = x_targets.split(';')
    alignments.append((eng_idx, mand_idx))


##### Load English and Mandarin text files

In [3]:
# Load the sentences from the English and Mandarin text files
with open('/Users/laurah/Desktop/PRINCETON/junior fall/IW/iw_2/en-zht.txt/GlobalVoices.en-zht.en', 'r', encoding='utf-8') as f:
    english_sentences = f.readlines()

with open('/Users/laurah/Desktop/PRINCETON/junior fall/IW/iw_2/en-zht.txt/GlobalVoices.en-zht.zht', 'r', encoding='utf-8') as f:
    mandarin_sentences = f.readlines()

In [12]:
english_sentences
df = pd.DataFrame(english_sentences, columns=['sentence'])
df['length'] = df['sentence'].apply(lambda x: len(x.split()))

# filter sentences with lengths between 4 and 30 inclusive
filtered_df = df[(df['length'] >= 6) & (df['length'] <= 30)]

# randomly sample 2117 sentences
sample_size = min(len(filtered_df), 2117)
random_sample = filtered_df.sample(n=sample_size, random_state=42)

# # extract the sentences back to a list if needed
sampled_sentences = random_sample['sentence'].tolist()

# # display sampled sentences
sampled_sentences

['In the meantime, farmers are reaping losses while pastoralists watch their cattle die. \n',
 'I have written a post on this significant step for Kenyan bloggers in giving the future of media in Africa. \n',
 'The main problem with the so-called "Green movement" is that they always try to clone the Islamic Revolution, or the Tunisian and Egyptian revolutions. \n',
 'Honestly, I felt, and still do, that life in Azerbaijan was more simple. \n',
 'In the 1920s a commune of volunteers from Czechoslovakia built a whole district of Bishkek. \n',
 'And this success of the Libyan Revolution comes at a very significant time in history. \n',
 'The new EPA is based on reciprocity - which means that for the first time, the playing field will be level. \n',
 'With our open southern border, there is little doubt that the Mexican violence will eventually spill over into Texas, resulting in innocent American deaths on American soil. \n',
 '@iyad_elbaghdadi: So far, four waves of airstrikes on Benghaz

In [14]:
random_sample[['sentence']].to_csv('global_voices_en_sentences.csv', index=False)

print("Sampled sentences have been saved to 'global_voices_en_sentences.csv'.")

Sampled sentences have been saved to 'global_voices_en_sentences.csv'.


##### Tokenize and POS-tag (English)

In [10]:
# Load spaCy model for English POS tagging
nlp_en = spacy.load('en_core_web_sm')

# Initialize the pkuseg model for Mandarin
# nlp_zht = stanza.Pipeline('zh-hans', processors='tokenize,pos')

# Tokenize and tag sentences
english_tags = [ [token.pos_ for token in nlp_en(sentence)] for sentence in english_sentences ]
# mandarin_tags = [ [word[1] for word in seg.cut(sentence)] for sentence in mandarin_sentences ]

##### Tokenize and POS-tag (Mandarin)

In [36]:
# Load spaCy model for Chinese POS tagging
nlp_zht = spacy.load("zh_core_web_sm")

# TEST - Sample sentence
# sentence = "我喜歡學習中文。"

mandarin_tags = []
for sentence in mandarin_sentences:
    doc = nlp_zht(sentence)
    sentence_tags = []
    
    for token in doc:
        sentence_tags.append(token.pos_)
    
    mandarin_tags.append(sentence_tags)

# Process the sentence
# doc = nlp_zht(sentence)

# for token in doc:
#     print(f"Token: {token.text}, POS: {token.pos_}, Detailed Tag: {token.tag_}")

In [17]:
english_tags_df = pd.DataFrame(english_tags)
english_tags_df.to_csv('english_tags.csv', index=False, header=False)

In [37]:
mandarin_tags_df = pd.DataFrame(mandarin_tags)
mandarin_tags_df.to_csv('mandarin_tags.csv', index=False, header=False)

In [20]:
len(english_tags_df)

144224

##### Identify number of unique POS tags

In [24]:
all_tags = pd.Series(english_tags_df.values.flatten())
unique_tags = all_tags.unique()
num_tags = len(unique_tags)
print("Number of unique POS tags:", num_tags)
print("Unique POS tags:", unique_tags)

Number of unique POS tags: 19
Unique POS tags: ['NUM' 'NOUN' 'ADP' 'PUNCT' 'SPACE' None 'DET' 'VERB' 'ADJ' 'PROPN'
 'CCONJ' 'SCONJ' 'AUX' 'ADV' 'PRON' 'PART' 'INTJ' 'X' 'SYM']


In [39]:
all_tags_zh = pd.Series(mandarin_tags_df.values.flatten())
unique_tags_zh = all_tags_zh.unique()
num_tags_zh = len(unique_tags_zh)
print("Number of unique Mandarin POS tags:", num_tags_zh)
print("Unique POS Mandarin tags:", unique_tags_zh)

Number of unique Mandarin POS tags: 17
Unique POS Mandarin tags: ['VERB' 'NOUN' 'PUNCT' 'SPACE' None 'PROPN' 'PART' 'PRON' 'ADV' 'NUM'
 'ADP' 'ADJ' 'X' 'DET' 'CCONJ' 'SCONJ' 'INTJ']


In [45]:
# Find the intersection
intersection = list(set(unique_tags) & set(unique_tags_zh))
print("Intersection:", intersection)
print(len(intersection))

# Find elements in unique_tags but not in the intersection
difference = list(set(unique_tags) - set(intersection))

# Print the result
print("Elements in unique_tags but not in intersection:", difference)

Intersection: [None, 'ADV', 'PUNCT', 'SPACE', 'DET', 'PRON', 'NUM', 'CCONJ', 'X', 'ADP', 'VERB', 'NOUN', 'ADJ', 'SCONJ', 'PART', 'INTJ', 'PROPN']
17
Elements in unique_tags but not in intersection: ['SYM', 'AUX']


##### Create One-Hot Encodings of POS Tags

In [69]:
# Reference to the one-hot vector positions
tag_order = []
for i, tag in enumerate(intersection):
    tag_order.append(tag)
print(tag_order)

[None, 'ADV', 'PUNCT', 'SPACE', 'DET', 'PRON', 'NUM', 'CCONJ', 'X', 'ADP', 'VERB', 'NOUN', 'ADJ', 'SCONJ', 'PART', 'INTJ', 'PROPN']


In [70]:
pos_to_index = {}
for i, tag in enumerate(unique_tags):
    pos_to_index[tag] = i

def pos_to_one_hot(pos_tag, pos_to_index, num_tags):
    one_hot = np.zeros(num_tags)
    index = pos_to_index[pos_tag]
    one_hot[index] = 1
    return one_hot

##### Convert Each Sentence's POS Tags to a Matrix

In [71]:
# ENGLISH
sentence_matrices_en = []

for sentence_tags in english_tags:
    one_hot_vectors_en = []

    for tag in sentence_tags:
        one_hot_vector = pos_to_one_hot(tag, pos_to_index, num_tags)
        one_hot_vectors_en.append(one_hot_vector)
        
    sentence_matrix = np.array(one_hot_vectors_en)
    sentence_matrices_en.append(sentence_matrix)

In [72]:
# MANDARIN
sentence_matrices_zh = []
for sentece_tags in mandarin_tags:
    one_hot_vectors_zh = []
    
    for tag in sentence_tags:
        one_hot_vector = pos_to_one_hot(tag, pos_to_index, num_tags)
        one_hot_vectors_zh.append(one_hot_vector)
    
    sentence_matrix = np.array(one_hot_vectors_zh)
    sentence_matrices_zh.append(sentence_matrix)

##### Create classification label for English/Mandarin

In [73]:
english_labels = [1] * len(sentence_matrices_en)
mandarin_labels = [0] * len(sentence_matrices_zh)

# combine vectors for rnn
all_sentences = sentence_matrices_en + sentence_matrices_zh
all_labels = english_labels + mandarin_labels

##### Convert to PyTorch Tensors; Prep to Feed into RNN

In [78]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

Using 'cpu' device


In [74]:
# Process Tensors for Input
class POSDataset(Dataset):
    def __init__(self, sentence_matrices, labels):
        # sentence matrices: list of numpy arrays, each representing a sentence with one-hot POS
        # labels: list of integers (1 for eng, 0 for zh)

        # Convert sentence matrices to PyTorch tensors
        sentence_tensors = []
        for matrix in sentence_matrices:
            sentence_tensor = torch.tensor(matrix, dtype=torch.float32)
            sentence_tensors.append(sentence_tensor)
        self.sentence_matrices = sentence_tensors

        # Convert labels to a 1D PyTorch tensor (flatten)
        label_list = []
        for label in labels:
            label_list.append(label)
        self.labels = torch.tensor(label_list, dtype=torch.long)

    # getter fct for length of sentence
    def __len__(self):
        return len(self.sentence_matrices)

    def __getitem__(self, index):
        return self.sentence_matrices[index], self.labels[index]

##### Apply the preparation function

In [81]:
dataset = POSDataset(all_sentences, all_labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Dataloader wraps an iterable around the dataset for improved access
# Create dataloaders from dataset
train_loader = DataLoader(train_dataset, batch_size = 1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=False)

##### RNN Model

In [82]:
# RNN Model Creation
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size, bias=False) # input to hidden
        self.h2h = nn.Linear(hidden_size, hidden_size) # hidden to hidden
        self.h2o = nn.Linear(hidden_size, output_size) # hidden to output
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x, hidden_state):
        x = self.i2h(x)
        hidden_state = self.h2h(hidden_state)
        hidden_state = torch.tanh(x + hidden_state)
        output = self.h2o(hidden_state)
        return self.softmax(output), hidden_state
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

##### Implement RNN Training

In [85]:
# initialize
input_size = all_sentences[0].shape[1] # num features per POS tag; length of one-hot vector, 17
hidden_size = 16 # num of hidden units
output_size = 2 # binary classification (1 for eng, 0 zh)

# create model, optimizer, and loss
model = RNN(input_size, hidden_size, output_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fct = nn.NLLLoss() # neg log likelihood loss fct

# train the function
def train_model(model, train_loader, epochs, optimizer, loss_fct):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            hidden = model.init_hidden().to(device)

            # zero gradients
            optimizer.zero_grad()

            # process each step in the seq
            loss = 0
            for i in range(X.size(1)):
                output, hidden = model(X[:, i, :], hidden)
                loss += loss_fct(output, y)
            
            # backprop + update weights
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 3) # gradient clipping for performance
            optimizer.step()

            total_loss += loss.item() / X.size(1)
            
        # Print epoch loss
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}')

##### Train model

In [86]:
train_model(model, train_loader, epochs=10, optimizer=optimizer, loss_fct=loss_fct)

RuntimeError: stack expects each tensor to be equal size, but got [20, 19] at entry 0 and [24, 19] at entry 1